In [27]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import sys; sys.path.insert(0, '../')
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd

pp = dict(surface='white', hemi='both')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Get Forward Model

In [30]:
info = get_info(kind='biosemi64')
fwd = create_forward_model(info=info, sampling='ico4')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    2.5s remaining:    4.3s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    2.6s remaining:    1.5s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.8s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.2s remaining:    0.5s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.3s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.4s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.5s remaining:    1.0s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.7s remaining:    0.4s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.8s finished


# Get sample data

In [33]:
settings = dict(number_of_sources=7, extents=(1, 50), duration_of_trial=0.025, target_snr=10e99)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

-- number of adjacent vertices : 5124
Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 1998.24it/s]


source data shape:  (5124, 25) (5124, 25)


100%|██████████| 2/2 [00:00<00:00, 332.33it/s]

Using control points [1.75265765e-08 2.31682157e-08 6.55735773e-08]


For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Regularization Optimizations

In [34]:
from invert.evaluate import corr, nmse
from invert.solvers.empirical_bayes import SolverChampagne

solver = SolverChampagne()
solver.make_inverse_operator(fwd, evoked, verbose=0)
stc_hat = solver.apply_inverse_operator(evoked)
stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))

print("Mean correlation: ", np.mean(corr(abs(stc_hat.data), abs(stc.data))))
print("Mean NMSE: ", np.mean(nmse(stc_hat.data, stc.data)))

Using control points [0.0000000e+00 0.0000000e+00 4.3726724e-07]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Mean correlation:  0.15963468528968133
Mean NMSE:  0.030273363524807778


In [5]:
from invert import Solver
from invert.evaluate import corr, nmse
from invert.adapters import contextualize_bd, focuss, s_focuss

solver_ = Solver(solver="laur")

solver_.make_inverse_operator(fwd, alpha="auto")
stc_hat = solver_.apply_inverse_operator(evoked)
# stc_hat.data[np.isnan(stc_hat.data)] = 0
stc_hat.plot(**pp, brain_kwargs=dict(title=solver_.name))
print("Mean correlation: ", np.mean(corr(stc_hat.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat.data, stc.data)))

stc_hat_sf = s_focuss(stc_hat, evoked, fwd, verbose=1)
stc_hat_sf.plot(**pp, brain_kwargs=dict(title=solver_.name + " shrinking FOCUSS"))
print("Mean correlation: ", np.mean(corr(stc_hat_sf.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat_sf.data, stc.data)))

stc_hat_sf = focuss(stc_hat, evoked, fwd, verbose=1)
stc_hat_sf.plot(**pp, brain_kwargs=dict(title=solver_.name + " FOCUSS"))
print("Mean correlation: ", np.mean(corr(stc_hat_sf.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat_sf.data, stc.data)))

# # Temporal Context
# stc_hat_cbd = contextualize_bd(stc_hat, fwd, fast=True, lstm_look_back=10, verbose=1)
# stc_hat_cbd.plot(**pp, brain_kwargs=dict(title="c"+solver_.name))

# print("Mean correlation: ", np.mean(corr(stc_hat_cbd.data, stc.data)))
# print("Mean NMSE: ", np.mean(nmse(stc_hat_cbd.data, stc.data)))

# # FOCUSS:
# stc_hat_focuss = focuss(stc_hat_cbd, evoked, fwd)
# stc_hat_focuss.plot(**pp, brain_kwargs=dict(title="c"+solver_.name+" FOCUSS"))

# print("Mean correlation: ", np.mean(corr(stc_hat_focuss.data, stc.data)))
# print("Mean NMSE: ", np.mean(nmse(stc_hat_focuss.data, stc.data)))


Using control points [1.00295540e-09 1.16641942e-09 2.91248426e-09]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Mean correlation:  0.14074433207742693
Mean NMSE:  0.023023068486939736
-- number of adjacent vertices : 1284
Shrinking FOCUSS:

Time step 0/10
Time step 1/10
Time step 2/10
Time step 3/10
Time step 4/10
Time step 5/10
Time step 6/10
Time step 7/10
Time step 8/10
Time step 9/10
Using control points [5.09529476e-59 1.25247081e-58 1.12653343e-55]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Mean correlation:  -0.003055457928142368
Mean NMSE:  0.005667790421865522
FOCUSS:

Time step 0/10
Iteration 0
Iteration 1
Iteration 2
converged at repeti

# Adapt

In [4]:
from invert.adapters import contextualize_bd, contextualize

stc_hat_cbd = contextualize_bd(stc_hat, fwd, lstm_look_back=10, num_units=128, num_epochs=50, verbose=0)
stc_hat_cbd.plot(**pp, brain_kwargs=dict(title="c"+solver.name))

print("Mean correlation: ", np.mean(corr(stc_hat_cbd.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat_cbd.data, stc.data)))

NameError: name 'stc_hat' is not defined

In [ ]:
from mne.minimum_norm import make_inverse_operator as mne_inverse
from mne.minimum_norm import apply_inverse as mne_apply
from mne import make_ad_hoc_cov
noise_cov = make_ad_hoc_cov(evoked.info, verbose=0)
mne_io = mne_inverse(evoked.info, fwd, noise_cov=noise_cov, fixed=True, loose=0, depth=0, verbose=0)
stc_hat = mne_apply(evoked, mne_io, method="MNE", verbose=0)
stc_hat.plot(**pp)